In [ ]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [2]:
df_news_clusterizada = pd.read_csv(config["DF_ITEMS_CLUSTERED_FEATURES"])

In [4]:
df_news_clusterizada.head()

,page,url,issued,modified,title,body,caption,cleaned_title,cleaned_caption,combined_text,cluster_pca,cleaned_body
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,caso bruno dom suspeito prisão temporária decr...,jeferson silva lima escoltado agentes polícia ...,caso bruno dom suspeito prisão temporária decr...,3,audiência custódia justiça amazonas decretou s...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...,linguajar santarenos diferenciado cheio identi...,expressões santarenas significam significam co...,linguajar santarenos diferenciado cheio identi...,6,vista aérea santarém ádrio ad produções paraen...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,shinzo abe morre baleado japão,atingido tiros espingarda caseira discursava c...,shinzo abe morre baleado japão atingido tiros ...,3,vídeo mostra assassino shinzo abe atirou costa...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,relator stf fachin vota marco temporal demarca...,ministro defendeu posse indígena diferente pos...,relator stf fachin vota marco temporal demarca...,4,relator stf fachin vota marco temporal demarca...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...",votos pedido vista suspende julgamento stf dem...,marco temporal índios reivindicar demarcação t...,votos pedido vista suspende julgamento stf dem...,6,pedido vista análise processo ministro alexand...


In [3]:
import numpy as np

# Convert the string column to datetime
df_news_clusterizada["data"] = pd.to_datetime(df_news_clusterizada["issued"])

In [4]:
df_news_clusterizada.head()

,Unnamed: 0,page,url,issued,modified,title,body,caption,cleaned_title,cleaned_caption,combined_text,cleaned_body,cluster,word_count,data
0,0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,caso bruno dom suspeito prisão temporária decr...,jeferson silva lima escoltado agentes polícia ...,caso bruno dom suspeito prisão temporária decr...,audiência custódia justiça amazonas decretou s...,2,704,2022-06-18 20:37:45+00:00
1,1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...,linguajar santarenos diferenciado cheio identi...,expressões santarenas significam significam co...,linguajar santarenos diferenciado cheio identi...,vista aérea santarém ádrio ad produções paraen...,0,728,2019-06-20 17:19:52+00:00
2,2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,shinzo abe morre baleado japão,atingido tiros espingarda caseira discursava c...,shinzo abe morre baleado japão atingido tiros ...,vídeo mostra assassino shinzo abe atirou costa...,2,1315,2022-07-08 08:55:52+00:00
3,3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,relator stf fachin vota marco temporal demarca...,ministro defendeu posse indígena diferente pos...,relator stf fachin vota marco temporal demarca...,relator stf fachin vota marco temporal demarca...,0,611,2021-09-09 19:06:46+00:00
4,4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...",votos pedido vista suspende julgamento stf dem...,marco temporal índios reivindicar demarcação t...,votos pedido vista suspende julgamento stf dem...,pedido vista análise processo ministro alexand...,0,506,2021-09-15 19:16:13+00:00


In [20]:


# Find the most recent date
most_recent_date = df_news_clusterizada["data"].max()

# Calculate the difference in days
df_news_clusterizada["idade_em_dias"] = (most_recent_date - df_news_clusterizada["data"]).dt.days

# Assign weights using exponential decay (you can adjust the base for scaling)
df_news_clusterizada["peso"] = np.exp(-df_news_clusterizada["idade_em_dias"] / 30)  # 30 days as the decay factor

In [21]:
most_recent_date

Timestamp('2022-08-15 02:57:16+0000', tz='UTC')

In [22]:
# Adjust range to [0.5, 1]
new_min = 0.1
new_max = 1
df_news_clusterizada["peso_ajustado"] = df_news_clusterizada["peso"] * (new_max - new_min) + new_min


In [23]:
df_news_clusterizada.head(50)

,Unnamed: 0,page,url,issued,modified,title,body,caption,cleaned_title,cleaned_caption,combined_text,cleaned_body,cluster,word_count,data,idade_em_dias,peso,peso_ajustado
0,0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,caso bruno dom suspeito prisão temporária decr...,jeferson silva lima escoltado agentes polícia ...,caso bruno dom suspeito prisão temporária decr...,audiência custódia justiça amazonas decretou s...,2,704,2022-06-18 20:37:45+00:00,57,1.495686e-01,0.234612
1,1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...,linguajar santarenos diferenciado cheio identi...,expressões santarenas significam significam co...,linguajar santarenos diferenciado cheio identi...,vista aérea santarém ádrio ad produções paraen...,0,728,2019-06-20 17:19:52+00:00,1151,2.175547e-17,0.100000
2,2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,shinzo abe morre baleado japão,atingido tiros espingarda caseira discursava c...,shinzo abe morre baleado japão atingido tiros ...,vídeo mostra assassino shinzo abe atirou costa...,2,1315,2022-07-08 08:55:52+00:00,37,2.913199e-01,0.362188
3,3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,relator stf fachin vota marco temporal demarca...,ministro defendeu posse indígena diferente pos...,relator stf fachin vota marco temporal demarca...,relator stf fachin vota marco temporal demarca...,0,611,2021-09-09 19:06:46+00:00,339,1.237292e-05,0.100011
4,4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...",votos pedido vista suspende julgamento stf dem...,marco temporal índios reivindicar demarcação t...,votos pedido vista suspende julgamento stf dem...,pedido vista análise processo ministro alexand...,0,506,2021-09-15 19:16:13+00:00,333,1.511232e-05,0.100014
5,5,a9fd6d34-6f40-4c90-849b-2ad36f04fd6f,http://g1.globo.com/politica/noticia/2021/08/2...,2021-08-27 18:49:03+00:00,2023-06-07 17:46:42+00:00,O que é o marco temporal sobre terras indígena...,Marco temporal sobre terras indígenas: entenda...,Tese analisada pelo Supremo após recurso da Fu...,marco temporal terras indígenas entenda jogo j...,tese analisada supremo recurso funai prevê ind...,marco temporal terras indígenas entenda jogo j...,marco temporal terras indígenas entenda julgad...,0,943,2021-08-27 18:49:03+00:00,352,8.021915e-06,0.100007
6,6,c2893f07-08fe-4b4c-8904-92f0fedc3040,http://g1.globo.com/pop-arte/noticia/2021/10/0...,2021-10-08 12:17:37+00:00,2023-06-07 23:40:04+00:00,"Shannen Doherty, de 'Barrados no Baile', compa...","Shannen Doherty, de 'Barrados no Baile', compa...","Nas redes sociais, atriz afirmou que usou 'o h...",shannen doherty barrados baile compartilha his...,redes sociais atriz afirmou usou humor ajudou ...,shannen doherty barrados baile compartilha his...,shannen doherty barrados baile compartilha his...,0,289,2021-10-08 12:17:37+00:00,31